# Assignment 2: Data Science in Python
## Daniel Stein 19201398

In [40]:
import urllib.request
import nltk
from requests import get
import requests
from bs4 import BeautifulSoup as BS
import seaborn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from bs4 import Comment
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tokenize = CountVectorizer().build_tokenizer()
from sklearn.feature_extraction import text
stopwords = text.ENGLISH_STOP_WORDS
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score, RepeatedKFold
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

# Task 1:Scrape reviews

For this step I chose to get Auto, Gym, and Hotel reviews.

In [41]:

# First we will store the three categories we want to extract in a list. 
# They are written as it is in the URL for any specific category.
# I also created empty lists where I will be storing all the links within the categories (1 for each category). 

rawhtml = ["automotive_list","gym_list","hotels_list"]
links_auto = []
links_gym = []
links_hotel = []

# I loop through the list of categories and add to the url where the review links are
for i in rawhtml:
#get content from url (elements)
    html_page = requests.get("http://mlg.ucd.ie/modules/yalp/"+i+".html")
    #nested structure for easy reading
    soup = BS(html_page.text, 'html.parser')
    
    #If statement to check and store for each category
    if i == "automotive_list":
        #go through part of element where we find links
        for link in soup.findAll('a'):
            #get link
            links_auto.append(link.get('href'))
        #remove first element as it is irrelevant
        links_auto.remove(links_auto[0])
# Repeat for the other 2 categories
    if i == "gym_list":
        for link in soup.findAll('a'):
                links_gym.append(link.get('href'))
        links_gym.remove(links_gym[0])
        
    if i == "hotels_list":
        for link in soup.findAll('a'):
                links_hotel.append(link.get('href'))
        links_hotel.remove(links_hotel[0])
    

In [42]:
# All the lists where the scores and comments will be stored
auto_score = []
auto_text = []
gym_score = []
gym_text = []
hotel_score = []
hotel_text = []

#loop through categories
for x in rawhtml:
    #if statement to separate categories
    if x == "automotive_list":
        #for each linked saved from the category url
        for y in links_auto:
            html_page = requests.get("http://mlg.ucd.ie/modules/yalp/"+y)
            soup = BS(html_page.text, 'html.parser')
            #extract from elements the rating part
            for link1 in soup.findAll('p', class_="rating"):
                link1 = link1.find('img')
                #give it a value of negative or positive depending on the stars
                if link1.get('alt') in ("1-star","2-star", "3-star"): 
                    link1 = "negative"
                else: 
                    link1 = "positive"
                #append to score list for the specific category
                auto_score.append(link1)
            #get text revies from each link
            for link2 in soup.findAll('p', class_="review-text"):
                #make into plain text.
                link2 = link2.text.strip()
                auto_text.append(link2)
        #Store in a dataframe both score and text
        df_auto = pd.DataFrame(list(zip(auto_score, auto_text)), columns = ['Score','Text'])    
#repeat for other 2 categories  
    if x == "gym_list":        
        for y in links_gym:
            html_page = requests.get("http://mlg.ucd.ie/modules/yalp/"+y)
            soup = BS(html_page.text, 'html.parser')
            for link1 in soup.findAll('p', class_="rating"):
                link1 = link1.find('img')
                if link1.get('alt') in ("1-star","2-star", "3-star"): 
                    link1 = "negative"
                else: 
                    link1 = "positive"
                gym_score.append(link1)
            for link2 in soup.findAll('p', class_="review-text"):
                link2 = link2.text.strip()
                gym_text.append(link2)
        df_gym = pd.DataFrame(list(zip(gym_score, gym_text)), columns = ['Score','Text'])
        
    if x == "hotels_list":    
        for y in links_hotel:
            html_page = requests.get("http://mlg.ucd.ie/modules/yalp/"+y)
            soup = BS(html_page.text, 'html.parser')
            for link1 in soup.findAll('p', class_="rating"):
                link1 = link1.find('img')
                if link1.get('alt') in ("1-star","2-star", "3-star"): 
                    link1 = "negative"
                else: 
                    link1 = "positive"
                hotel_score.append(link1)
            for link2 in soup.findAll('p', class_="review-text"):
                link2 = link2.text.strip()
                hotel_text.append(link2)
        df_hotel = pd.DataFrame(list(zip(hotel_score, hotel_text)), columns = ['Score','Text'])

# Task 2: Preprocessing

As seen in the first cell of code, we have created a list containing stopwords using sklearn. Below is an extention of words or numbers that are irrelevant when classifying, and that might actually negatively affect the results. I created this list after running the classification and displaying the terms. 

In [43]:
stopwords_num =('00', '000', '10', '100', '101', '10am', '11', '110', '11am', '12',
                '13', '14', '15', '150', '16', '17', '18', '19', '1am', '1st', '20', 
                '200', '2009', '2010', '2012', '2013', '2014', '2015', '2016', '2017', 
                '22', '24', '25', '2am', '2nd', '2pm', '30', '300', '30am', '30pm', 
                '30th', '35', '39', '3am', '3pm', '3rd', '40', '400', '45', '48','4pm',
                '4th', '50', '500', '55', '5pm', '5th', '60', '600', '65', '6am', '70',
                '75', '80', '800', '8am', '8th', '90', '95', '99', '9am','3000', '32',
                '350', '450', '4runner', '5000', '59', '6pm', '700', '750', '7am', '900', '9am'
               '00am', '00pm', '02', '06', '07', '08', '09', '1000', '105', '10pm', '10th', '115',
                '11pm', '120', '1200', '125', '12pm', '130', '15th', '160', '16th', '180', '1pm',
                '2005', '2007', '2008', '2011', '2018', '202', '21', '215', '23', '250', '26', '260',
                '27', '28', '29', '30mins', '33', '34', '35th', '36', '360', '38', '3x', '43', '458',
                '45am', '47','49', '52', '5am', '63', '67', '69', '72', '77', '79', '7pm', '7th',
                '80s', '85', '89', '8pm', '98', '9pm','00am','140', '1500', '170', '1800', '2000', '2003', '2004', '2006')

Below are the steps for preprocessing all the text reviews. I made them all into lower case and remove all stop words identified by the two lists mentioned before. I then detokenized the text and replaced it in the dataframe according to the row number named "count". I repeated this step for all the dataframes

In [44]:
#preprocessing
#create a count that will increase as loop runs
count = 0
#go through dataframe text column
for i in df_auto['Text']:
    #lower case the text
    lower = i.lower()
    #tokenize text and make variable
    token = tokenize(lower)    
    stop_rm = []
    for tokens in token:
        if not tokens in stopwords and not tokens in stopwords_num:
            stop_rm.append(tokens)
    text = TreebankWordDetokenizer().detokenize(stop_rm)
    df_auto.iloc[count,1] = text
    count += 1
    


    
    

In [45]:
#preprocessing
#create a count that will increase as loop runs
count = 0
#go through dataframe text column
for i in df_gym['Text']:
    #lower case the text
    lower = i.lower()
    #tokenize text and make variable
    token = tokenize(lower)    
    stop_rm = []
    for tokens in token:
        if not tokens in stopwords and not tokens in stopwords_num:
            stop_rm.append(tokens)
    text = TreebankWordDetokenizer().detokenize(stop_rm)
    df_gym.iloc[count,1] = text
    count += 1


In [46]:
#preprocessing
#create a count that will increase as loop runs
count = 0
#go through dataframe text column
for i in df_hotel['Text']:
    #lower case the text
    lower = i.lower()
    #tokenize text and make variable
    token = tokenize(lower)    
    stop_rm = []
    for tokens in token:
        if not tokens in stopwords and not tokens in stopwords_num:
            stop_rm.append(tokens)
    text = TreebankWordDetokenizer().detokenize(stop_rm)
    df_hotel.iloc[count,1] = text
    count += 1

I created one last dataframe with all the data from the three previously processed dataframes. This holds all the information for all categories. I will later show why I did this and how this dataframe was used.

In [47]:
df_all = pd.concat([df_auto,df_gym,df_hotel])
    

# Task 2 :Classification and Evaluation

In this task, I will be using the Random Forest Classifier with an n_estimator of 100 (number of trees in the forest). 

### Auto Reviews

Below we can see for the auto dataset, that there are 2000 documents and 2449 distinct terms in those documents. While the number of document is always the same across all datasets, the number of terms tend to differ.

In [73]:
#Here I create variables containing each field of the dataframes that will be separated and used in the training and testing process
document_auto = df_auto['Text']
target_auto = df_auto['Score']

#We creare a vectorizer
vectorizer =CountVectorizer(min_df = 5)
#We create the table of words it will show the number of documents and distinct terms
X_auto = vectorizer.fit_transform(document_auto)
print(X_auto.shape)

(2000, 2449)


In [74]:
#Here we create a list that holds all the distinct terms (note that X holds numerical values and terms holds written words)
terms = vectorizer.get_feature_names()
print("Vocabulary has %d distinct terms" % len(terms))

Vocabulary has 2449 distinct terms


Below is an extract of the first 50 terms from the Auto dataframe, we can see that numbers were removed (numbers are always first) in the preprocessing steps. We also see there are a few words that tend to be neutral and not holding any sentiment such as names. We could create more lists or even download one so that we could only keep words expressing sentiment. I decided not to apply this step, as names will be rather rare and overall I believe this words will not skew the results.

In [75]:
# Display a set of sample terms, I used this to verify if there was a cluster of words or numbers that I could
#then put in a seperate stopword list that was used in the preprocessing step
print(terms[0:50])

['aa', 'aaa', 'able', 'absolute', 'absolutely', 'ac', 'accent', 'accept', 'accident', 'accommodate', 'accommodating', 'accord', 'according', 'account', 'accurate', 'ace', 'actual', 'actually', 'acura', 'ad', 'adam', 'add', 'added', 'adding', 'addition', 'additional', 'address', 'addressed', 'adjusted', 'adjuster', 'admit', 'admitted', 'advance', 'advantage', 'advertise', 'advertised', 'advice', 'advise', 'advised', 'advisor', 'afford', 'affordable', 'afraid', 'aftermarket', 'afternoon', 'age', 'agent', 'ago', 'agree', 'agreed']


We can now create the variables for the training and test data. Below we specify what values we are using and how much of the data we are using for training and testing: I decided to use 70% for training and 30% for testing. I think these ratios are balanced, and I will avoid over-fiiting and underfitting my classifier. 

In [76]:
#Here we stablish our training and test data and results and divide it into 70/30 for training/test.
data_train, data_test, target_train, target_test = train_test_split(X_auto, target_auto, test_size=0.30)
print("Training set has %d examples" % data_train.shape[0] )
print("Test set has %d examples" % data_test.shape[0] )

Training set has 1400 examples
Test set has 600 examples


In [77]:
#This is our classifier.
clf=RandomForestClassifier(n_estimators=100)

In [78]:
#Here we run the classifier using the data provided above and store the results.
clf.fit(data_train,target_train)
predict = clf.predict(data_test)

After running the classifier we can store the results, and compare our predicted data to the actual results in order to get an accuracy score. Below we see the accuracy for a single run (this will number normally change as we do more runs), and it retains an accuracy of between 80% to 90% in almost most cases. This shows that our classifier is correctly classifying text into positive or negative with high accuracy (considering we are dealing with sentiment and text).

In [79]:
#Here we get the accuracy of our classifier by taking the predicted values and comparing with the actual results.
print("The accuracy is:", accuracy_score(target_test, predict))

The accuracy is: 0.8716666666666667


Now that we have the accuracy of a single run, we can use more data to evaluate our results. Below we see the confusion matrix for our results. In this matrix our horizontal values are predicted results while our vertical are actual results. From this we can see that in the majority ofthe cases, negative reviews were incorrectly claffied more often than positive ones. We can also see that there are overall more positive reviews in our data than negative ones, this reflects on our matrix as most of the time we will get the sum of the lowe cells to be much higher than the sum of the top cells (in this test data most of the time there are also more positive than negative). 

We can see on the report bellow the different statistics form our classifier:
Precision: of the reviews identified as either positive or negative, how many were actually correct. 
Recall: of all the positive and negative how many were identified correctly.
F1-score: This shows the balance between precision and recall.

From the statistics we can note a few things:

Negative: Overall precision on negative reviews is rather high, this means that of the reviews classified as negative, most of them were actual negatives. As for recall, we see that it is rather low in comparison with the other statistics, this means that of all the actual negative reviews not that many of them were classified as negative, this brings down the balance of f1-score, and overall brings down accuracy for our classifier.

Positive: For the positive reviews we see that in general it scores well in precision, the fact that it scored so high in precision even if we know that a lot of negative were predicted as positive is due to how many more positive reviews there are compared to negative. As we can see recall has a very high score, which means of all the positive reviews most of them were classified correctll. The combination of these two give a high f1-score.



In [80]:
#Create a confusion matrix and further evaluation results.
cm = confusion_matrix(target_test, predict,labels=['negative','positive'])
print(cm)
print("\n")
print(classification_report(target_test, predict, target_names=["negative","positive"]))

[[184  53]
 [ 24 339]]


              precision    recall  f1-score   support

    negative       0.88      0.78      0.83       237
    positive       0.86      0.93      0.90       363

    accuracy                           0.87       600
   macro avg       0.87      0.86      0.86       600
weighted avg       0.87      0.87      0.87       600



Lastly, we can do a cross validation evaluation, which will help avoid any over-fitting.

Cross validation runs our classifier a specific number of times and divides the training and testing data differently each time. By doing so, come across as many posible scenarios as possible.

In [93]:
#cross validation using 10 folds which will give 10 accuracies that will be stored in a list.
cross_val_auto =  cross_val_score(clf,X_auto, target_auto, cv=10)
cross_val_auto

array([0.90547264, 0.90049751, 0.93      , 0.875     , 0.855     ,
       0.81      , 0.825     , 0.855     , 0.87437186, 0.89447236])

With cross validation, we can find an average accuracy that is closer to actual accuracy together with standard deviation.

In [103]:
#Average and standard deviation from cross validation
print("Accuracy: %0.2f (+/- %0.2f)" % (cross_val_auto.mean(), cross_val_auto.std() * 2))

Accuracy: 0.87 (+/- 0.07)


We can repeat the same process for all categories and analyze the classifier when used on each one.

### Gym Reviews

Now we will apply the same steps for the gym reviews dataset.

Below, we see the number of documents is again 2000 while the number of terms increase to 2799.

We can also see that again there are in average more positive reviews in the test data (confusion matrix) than negative reviews, and more incorrectly identified negative reviews. This follows the same pattern as the previews dataset.

We can also observe a similar pattern with the statistics as the previous dataset for both negative and positive, which could be because both datasets contain more positive than negative reviews. Which means positive data has more scenarios covered and so the classifier performs when classifying positive reviews.

In [83]:
document_gym = df_gym['Text']
target_gym = df_gym['Score']



X_gym= vectorizer.fit_transform(document_gym)
print(X_gym.shape)
terms = vectorizer.get_feature_names()

data_train, data_test, target_train, target_test = train_test_split(X_gym, target_gym, test_size=0.30)

clf.fit(data_train,target_train)
predict = clf.predict(data_test)
cm = confusion_matrix(target_test, predict,labels=['negative','positive'])
print(cm)
print("The accuracy is:", accuracy_score(target_test, predict))
print("\n")
print(classification_report(target_test, predict, target_names=["negative","positive"]))

(2000, 2799)
[[141  57]
 [ 27 375]]
The accuracy is: 0.86


              precision    recall  f1-score   support

    negative       0.84      0.71      0.77       198
    positive       0.87      0.93      0.90       402

    accuracy                           0.86       600
   macro avg       0.85      0.82      0.83       600
weighted avg       0.86      0.86      0.86       600



Lastly, I did another cross validation for this model showing the average and standard deviation.

In [84]:
cross_val_gym =  cross_val_score(clf,X_gym, target_gym, cv=10)
cross_val_gym

array([0.85074627, 0.895     , 0.9       , 0.88      , 0.86      ,
       0.905     , 0.905     , 0.875     , 0.88      , 0.85427136])

In [104]:
print("Accuracy: %0.2f (+/- %0.2f)" % (cross_val_gym.mean(), cross_val_gym.std() * 2))

Accuracy: 0.88 (+/- 0.04)


For our last category, we can see that in average it performs quite well in comparison with the previous dataset trained clasifiers. This is bery likely because of all our datasets, this one holds a more balanced distribution of positive to negative results. Ending as shown below, with a higher precision and recal for negative reviews.

In [106]:
document_hotel = df_hotel['Text']
target_hotel = df_hotel['Score']


X_hotel = vectorizer.fit_transform(document_hotel)


terms = vectorizer.get_feature_names()
data_train, data_test, target_train, target_test = train_test_split(X_hotel, target_hotel, test_size=0.30)

clf.fit(data_train,target_train)
predict = clf.predict(data_test)

cm = confusion_matrix(target_test, predict,labels=['negative','positive'])
print(cm)
print("The accuracy is:", accuracy_score(target_test, predict))
print("\n")
print(classification_report(target_test, predict, target_names=["negative","positive"]))

[[198  51]
 [ 12 339]]
The accuracy is: 0.895


              precision    recall  f1-score   support

    negative       0.94      0.80      0.86       249
    positive       0.87      0.97      0.91       351

    accuracy                           0.90       600
   macro avg       0.91      0.88      0.89       600
weighted avg       0.90      0.90      0.89       600



Below we have again a cross validiation for hotel reviews dataset. As we can see the average accuracy is in the same range as the previous two dataset classifiers.

In [86]:
cross_val_hotel =  cross_val_score(clf,X_hotel, target_hotel, cv=10)
cross_val_hotel

array([0.90049751, 0.87064677, 0.85572139, 0.90547264, 0.88      ,
       0.84      , 0.85427136, 0.88442211, 0.82914573, 0.8241206 ])

In [91]:
print("Accuracy: %0.2f (+/- %0.2f)" % (cross_val_hotel.mean(), cross_val_hotel.std() * 2))

Accuracy: 0.86 (+/- 0.05)


I thought it would be interesting to join all the datasets into one and train/test our classifier using all the reviews. Below are the statistics, and as we can see it performed very well in all parts. 

In [87]:
document_all = df_all['Text']
target_all = df_all['Score']


X_all = vectorizer.fit_transform(document_all)

terms = vectorizer.get_feature_names()
data_train, data_test, target_train, target_test = train_test_split(X_all, target_all, test_size=0.30)

clf.fit(data_train,target_train)
predict = clf.predict(data_test)

print(cm)
print("The accuracy is:", accuracy_score(target_test, predict))
cm = confusion_matrix(target_test, predict,labels=['negative','positive'])
print("\n")
print(classification_report(target_test, predict, target_names=["negative","positive"]))


[[208  41]
 [ 13 338]]
The accuracy is: 0.8988888888888888


              precision    recall  f1-score   support

    negative       0.88      0.84      0.86       659
    positive       0.91      0.94      0.92      1141

    accuracy                           0.90      1800
   macro avg       0.89      0.89      0.89      1800
weighted avg       0.90      0.90      0.90      1800



The cross validation below shows the average (one of the highest) of accuracy for this data.

In [90]:
cross_val_all =  cross_val_score(clf,X_all, target_all, cv=10)
cross_val_all

array([0.8968386 , 0.85524126, 0.86189684, 0.88333333, 0.87333333,
       0.88      , 0.89333333, 0.90651085, 0.84974958, 0.88146912])

In [92]:
print("Accuracy: %0.2f (+/- %0.2f)" % (cross_val_all.mean(), cross_val_all.std() * 2))

Accuracy: 0.88 (+/- 0.03)


I also wanted to show the number of positive and negative reviews in our three datasets.

As we can see, for all of the datasets there are more positive reviews than negatives, with hotel being the most balanced. This bias in our data could make it difficult to train our classifier to detect negative reviews, and as shown above this is precisely what happened. For all of our datasets, the recal for our negative reviews were muck lower than the positives.

Another factor that I believed affected our outcome was the manner in which we divided star based ratings into positive and negative. If we read the reviews for three stars, we could debate wether the review was a positive one or negative. This makes it so many of the previously rated 3-stars are classified as positive. I would consider making 3-stars into positive or a third class "neutral". The issue that might raise from doing this is having even less negative reviews in our data. 

In [100]:
df_auto.describe()

,Score,Text
count,2000,2000
unique,2,2000
top,positive,got sent dealership doing repair work mercedes...
freq,1212,1


In [101]:
df_gym.describe()

,Score,Text
count,2000,2000
unique,2,2000
top,positive,son wanted try jujitsu location highly recomme...
freq,1299,1


In [102]:
df_hotel.describe()

,Score,Text
count,2000,2000
unique,2,2000
top,positive,location sweet away minutes busy las vegas str...
freq,1176,1


# Task 3: Performance

For this task we are going to train our classifier using one entire dataset, and test it on the other two. I expect it to show a similar but slightly lower accuracy, as certain terms and words that only apply for a specific category.

### Auto dataset trained classifier on Gym and Hotel data

We can see below that in average the auto dataset trained classifier performed well in predicting positive and negative reviews at around 85% accuracy. We can also observe similar patterns seen before of a lower recall on negative than positve and around the same precision on both. This is mainly due to the disparity in total negative against positive reviews in our data and the fact that many "negative" reviews hold positive terms (3-star reviews). As a result her and throughout all of the evaluations, negative recall has been significantly lower than positive, as of all the actual negative reviews a lower portion were classified as negative.

In [122]:
#create variables with all texts and scores from our datasets.
document_auto = df_auto['Text']
target_auto = df_auto['Score']
document_gym = df_gym['Text']
target_gym = df_gym['Score']
document_hotel = df_hotel['Text']
target_hotel = df_hotel['Score']

#create vectors and store separately for all of our text
X_auto = vectorizer.fit_transform(document_auto)
X_gym = vectorizer.transform(document_gym)
X_hotel = vectorizer.transform(document_hotel)

#set the train data as the vector for auto
data_train = X_auto
#set the train target as the auto target
target_train =  target_auto
#use gym as our test data and target
data_test = X_gym
target_test =  target_gym
#run the classifier on our training data
clf.fit(data_train,target_train)
#run model on our test data
predict = clf.predict(data_test)

#print accuracy, confusion matrix, and classification report
print("The accuracy of using Auto model on Gym data is:", accuracy_score(target_test, predict))
print("\n")
cm = confusion_matrix(target_test, predict,labels=['negative','positive'])
print(cm)
print("\n")
print(classification_report(target_test, predict, target_names=["negative","positive"]))
#repeat for hotel data as test data

The accuracy of using Auto model on Gym data is: 0.8555


[[ 512  189]
 [ 100 1199]]


              precision    recall  f1-score   support

    negative       0.84      0.73      0.78       701
    positive       0.86      0.92      0.89      1299

    accuracy                           0.86      2000
   macro avg       0.85      0.83      0.84      2000
weighted avg       0.85      0.86      0.85      2000



In [123]:
data_test = X_hotel
target_test =  target_hotel
predict = clf.predict(data_test)
print("The accuracy of using Auto model on Hotel data is:", accuracy_score(target_test, predict))
print("\n")
cm = confusion_matrix(target_test, predict,labels=['negative','positive'])
print(cm)
print("\n")
print(classification_report(target_test, predict, target_names=["negative","positive"]))
#Repeat switching around our train data.

The accuracy of using Auto model on Hotel data is: 0.8535


[[ 659  165]
 [ 128 1048]]


              precision    recall  f1-score   support

    negative       0.84      0.80      0.82       824
    positive       0.86      0.89      0.88      1176

    accuracy                           0.85      2000
   macro avg       0.85      0.85      0.85      2000
weighted avg       0.85      0.85      0.85      2000



### Gym dataset trained classifier on Auto and Hotel data

Next, we can see that the accuracy of our Gym trained model is slightly lower when predicting reviews. Following a similar pattern with precision and recall. However, we can see that negative reviews statistics underperformed by a significant amount. This might be due to the gym dataset being the one with the fewer amount of negative reviews. Which means even less negative scenarios covered in the training process. This had an obvious result on all negative review statistics.

In [118]:

X_auto = vectorizer.fit_transform(document_auto)
X_gym = vectorizer.transform(document_gym)
X_hotel = vectorizer.transform(document_hotel)

data_train = X_gym
target_train =  target_gym
data_test = X_auto
target_test =  target_auto
clf.fit(data_train,target_train)
predict = clf.predict(data_test)


print("The accuracy of using Gym model on Auto data is:", accuracy_score(target_test, predict))
print("\n")
cm = confusion_matrix(target_test, predict,labels=['negative','positive'])
print(cm)
print("\n")
print(classification_report(target_test, predict, target_names=["negative","positive"]))


The accuracy of using Gym model on Auto data is: 0.803


[[625 163]
 [231 981]]


              precision    recall  f1-score   support

    negative       0.73      0.79      0.76       788
    positive       0.86      0.81      0.83      1212

    accuracy                           0.80      2000
   macro avg       0.79      0.80      0.80      2000
weighted avg       0.81      0.80      0.80      2000



In [119]:
data_test = X_hotel
target_test =  target_hotel
predict = clf.predict(data_test)
print("The accuracy of using Gym model on Hotel data is:", accuracy_score(target_test, predict))
print("\n")
cm = confusion_matrix(target_test, predict,labels=['negative','positive'])
print(cm)
print("\n")
print(classification_report(target_test, predict, target_names=["negative","positive"]))

The accuracy of using Gym model on Hotel data is: 0.829


[[ 642  182]
 [ 160 1016]]


              precision    recall  f1-score   support

    negative       0.80      0.78      0.79       824
    positive       0.85      0.86      0.86      1176

    accuracy                           0.83      2000
   macro avg       0.82      0.82      0.82      2000
weighted avg       0.83      0.83      0.83      2000



### Hotel dataset trained classifier on Auto and Gym data

Lastly, we see the performance of the hotel trained classifier on the other two dataset. It follows a similar pattern to the Auto trained model with a pretty high accuracy, precision and recall. This might be because both the hotel dataset and the auto dataset have a more balanced number of positivie reviews to negative reviews, which means the classifier is better trained to predict negative reviews, while doing a great job in predicting positive ones.

In [120]:

X_auto = vectorizer.fit_transform(document_auto)
X_gym = vectorizer.transform(document_gym)
X_hotel = vectorizer.transform(document_hotel)

data_train = X_hotel
target_train =  target_hotel
data_test = X_auto
target_test =  target_auto
clf.fit(data_train,target_train)
predict = clf.predict(data_test)


print("The accuracy of using Hotel model on Auto data is:", accuracy_score(target_test, predict))
print("\n")
cm = confusion_matrix(target_test, predict,labels=['negative','positive'])
print(cm)
print("\n")
print(classification_report(target_test, predict, target_names=["negative","positive"]))


The accuracy of using Hotel model on Auto data is: 0.8635


[[ 685  103]
 [ 170 1042]]


              precision    recall  f1-score   support

    negative       0.80      0.87      0.83       788
    positive       0.91      0.86      0.88      1212

    accuracy                           0.86      2000
   macro avg       0.86      0.86      0.86      2000
weighted avg       0.87      0.86      0.86      2000



In [121]:
data_test = X_gym
target_test =  target_gym
predict = clf.predict(data_test)
print("The accuracy of using Hotel model on Gym data is:", accuracy_score(target_test, predict))
cm = confusion_matrix(target_test, predict,labels=['negative','positive'])
print(cm)
print("\n")


print(classification_report(target_test, predict, target_names=["negative","positive"]))

The accuracy of using Hotel model on Gym data is: 0.867
[[ 537  164]
 [ 102 1197]]


              precision    recall  f1-score   support

    negative       0.84      0.77      0.80       701
    positive       0.88      0.92      0.90      1299

    accuracy                           0.87      2000
   macro avg       0.86      0.84      0.85      2000
weighted avg       0.87      0.87      0.87      2000



In the end, we can see that the model trained with the gym dataset perform poorly in comparison to the other models in particular when predicting negative reviews. It might be interesting to train our data with two dataset and test on the last one. I believe this will better train our model on negative reviews and give an over-all better classification results.

# Conclusion

In conclusion I chose to use this style of evaluation because it represents with rations and numerical values how well our classifier performed. While we could have used accuracy alone, using recall and precision calculations give further depth to our results and show how over-all our models did better when predicting positive reviews (due to having many more positive reviews in our data). From the evaluation done above, I beleve that my classification model performed well and predicted accurately in all the scenarios. 

A few changes I would sugest for future analysis would be to separate the ratings into more classes as a 3-star could be taken a positive or negative. And perhaps work with more data as we have seen the amount of negative reviews was minimal compared to our positive reviews, which as shown, affected the performance of our classifier.